# Example of HTML capabilities of project inside IPython notebook

## Initializing Environment

In [1]:
from openerp_proxy.session import IPYSession as Session
from openerp_proxy.core import Client
from openerp_proxy.ext.all import HField
import openerp_proxy.plugins.module_utils  # Enable module_utils plugin

# connect to local instance of server
cl = Client('localhost')

# create demo database
cl.services.db.create_db('admin', 'demo_db_1', demo=True, lang='en_US')

# login to created database
ldb = cl.login('demo_db_1', 'admin', 'admin')  # all this arguments could be passed directly to Client constructor.

# Note that both 'cl' and 'ldb' are instances of same class
# the difference is in presense of database connection args.
# cl have no them, but ldb have it.
assert(type(cl) == type(ldb))

# and let's look how it is displayed in IPython
ldb

Host,localhost
Port,8069
Protocol,xml-rpc
Database,demo_db_1
login,admin


## Session class

If You often need to connect to same databases, there are a ```openerp_proxy.session.Session``` class, which automaticaly save, most of your connections, made via ```Session.connect``` method in specified file.
In our case we will use ```openerp_proxy.session.IPYSession``` class, which have better IPython representation.

In [11]:
# create session instance
session = Session('~/.openerp_proxy.local.json')  # default file path is '~/.openerp_proxy.json'

And there are option You may be interested in. It is 'store_passwords', which automaticaly saves password You have used for connection.

In [12]:
session.option('store_passwords', True)

True

We use ```openerp_proxy.core.Client``` class to create connection to database, so our session does not know anything about it. Let's add it to our session:

In [13]:
session.add_db(ldb)
session

DB URL,DB Index,DB Aliases
xml-rpc://admin@localhost:8069/demo_db_1,1,


And now we clud get this connection from session by index, or by URL (look at the table above). But to simplify next connections, we may add aliase to this one connection

In [14]:
session.aliase('ldb', ldb)
session

DB URL,DB Index,DB Aliases
xml-rpc://admin@localhost:8069/demo_db_1,1,ldb


So, now, to get connection again we could just type ```session.ldb```.

Ok. initialization is done, and now we could save it. (NOTE, that session will be automatically saved when its destructor called, so in most cases, it will be saved automaticaly

In [15]:
session.save()

## Connect to odoo database

Let's now create new instance of session, and connect to created above database

In [2]:
session = Session('~/.openerp_proxy.local.json')  # default file path is '~/.openerp_proxy.json'
ldb = session.ldb
ldb

Host,localhost
Port,8069
Protocol,xml-rpc
Database,demo_db_1
login,admin


## Module Utils plugin

Our database is clean, for next code, we need to install ```sale``` addon. For this, we have ```module_utils``` plugin, out-of-the box, which simplyfies work with modules. To enable this plugin, we just need to import it, and then we will have it in ```ldb.plugins``` property. This plugin was imported above.
This plugin extends ```ir.module.module``` model from client side, adding simple methods: ```install``` and ```upgrade``` to it for shorter syntax. If You're interested for code, look [here](https://github.com/katyukha/openerp-proxy/blob/master/openerp_proxy/plugins/module_utils.py), it is very simple!

So now, let's install ```sale``` module.  (Note that in most cases, **IPython autocompletition** work's fine)

In [3]:
ldb.plugins.module_utils.m_sale.install()

{'auto_refresh': 0,
 'auto_search': True,
 'context': {'disable_log': True},
 'domain': False,
 'filter': False,
 'groups_id': [],
 'help': False,
 'id': 291,
 'limit': 80,
 'multi': False,
 'name': 'Configure Accounting Data',
 'nodestroy': False,
 'res_id': 0,
 'res_model': 'account.installer',
 'search_view': '{\'name\': \'default\', \'fields\': {\'date_stop\': {\'selectable\': True, \'required\': True, \'type\': \'date\', \'string\': \'End Date\', \'views\': {}}}, \'arch\': \'<search string="account.installer"><field name="date_stop" modifiers="{&quot;required&quot;: true}"/></search>\', \'model\': \'account.installer\', \'type\': \'search\', \'view_id\': 0, \'field_parent\': False}',
 'search_view_id': False,
 'src_model': False,
 'target': 'new',
 'type': 'ir.actions.act_window',
 'usage': False,
 'view_id': [474, 'account.installer.form'],
 'view_ids': [],
 'view_mode': 'form',
 'view_type': 'form',
 'views': [[474, 'form']]}

Congratulation! module was installed! Now we need only to refresh caches, to see new models in database.

In [3]:
ldb.clean_caches()

## Get object / model

*ldb* here represents database connection (*Client* class instance)
As told in help message above *.get_obj* method allows to get instance of specified
*Object* proxy, where Object means *model*, *document*.

In [4]:
so = ldb.get_obj('sale.order')
so

Name,Sales Order
Proxy,xml-rpc://admin@localhost:8069/demo_db_1
Model,sale.order
Record count,8


Also it is posible to use shorter (dictionary style) syntax:

In [5]:
so = ldb['sale.order']
so

Name,Sales Order
Proxy,xml-rpc://admin@localhost:8069/demo_db_1
Model,sale.order
Record count,8


And as result of using [*openerp_proxy.ext.sugar*](http://pythonhosted.org/openerp_proxy/module_ref/openerp_proxy.ext.html#module-openerp_proxy.ext.sugar) extension
(which is automaticaly imported in 'openerp_proxy.ext.all')
there are attribute-style access (which also support's IPython auto-completition):

In [6]:
so = ldb._sale_order
so

Name,Sales Order
Proxy,xml-rpc://admin@localhost:8069/demo_db_1
Model,sale.order
Record count,8


## Getting information about available columns for Object

In [7]:
so.columns_info

name,info.string,info.type,info.selection,info.help,info.readonly,info.required,info.relation_field,info.select
amount_tax,Taxes,float,,The tax amount.,1,None,None,None
amount_total,Total,float,,The total amount.,1,None,None,None
amount_untaxed,Untaxed Amount,float,,The amount without tax.,1,None,None,None
client_order_ref,Customer Reference,char,,None,None,None,None,None
company_id,Company,many2one,,None,True,None,None,None
create_date,Creation Date,datetime,,Date on which sales order is created.,True,None,None,True
currency_id,Currency,many2one,,None,True,True,None,None
date_confirm,Confirmation Date,date,,Date on which sales order is confirmed.,True,None,None,True
date_order,Date,date,,None,True,True,None,True
fiscal_position,Fiscal Position,many2one,,None,None,None,None,None


## Search for sale orders

In [8]:
# Standard search .search_records(domain)
so_list = so.search_records([])
so_list

Object,Object ('sale.order')
Proxy,xml-rpc://admin@localhost:8069/demo_db_1
Record count,8


Also there are shorter syntax provided by [*openerp_proxy.ext.sugar*](http://pythonhosted.org/openerp_proxy/module_ref/openerp_proxy.ext.html#module-openerp_proxy.ext.sugar) extension:

In [9]:
so_list = so([])
so_list

Object,Object ('sale.order')
Proxy,xml-rpc://admin@localhost:8069/demo_db_1
Record count,8


## Display Sale orders as HTML table

It is posible to represent RecordList as HTML table, with ability to highlight rows by specified conditions.
Also, when building result table, it is posible to display values of related fields, and even method calls. This functionality is implemented in [*openerp_proxy.ext.repr*](http://pythonhosted.org/openerp_proxy/module_ref/openerp_proxy.ext.html#module-openerp_proxy.ext.repr) module

In [10]:
# High light rows by condition
highlighters = {
    '#99FF99': lambda x: x.state == 'done',
    '#9999FF': lambda x: x.state == 'draft',
    '#FFFF99': lambda x: x.state == 'progress',
}

# Display as table.
# Note that prefetch method is used to fetch some set of fields via one RPC call.
# on big datasets it may speed up performance signifiantly.
# Each RecordList instance have related cache, which reduce need of reading data on each field get.
so_list.prefetch('id', 'name', 'partner_id', 'partner_id.email', 'state')
so_table = so_list.as_html_table(
    'id',
    'name',
    # _name attribute provides result of *name_search method:
    HField('partner_id._name', name='Partner name'),
    # silent=True means, if field cannot be found, not throw error
    HField('partner_id.email', name='Partner email', silent=True),
    # Also it is posible to display result of method calls
    # 'as_html_list()' is method of RecordList.
    'order_line.as_html_list',
    # Argument can be passed as tuple, (field, field name)
    ('invoice_ids.as_html_list', 'Related Invoices'),
    'state',
    highlighters=highlighters,
)
so_table

id,name,Partner name,Partner email,order_line.as_html_list,Related Invoices,state
8,SO008,Millennium Industries,False,"20: Laptop Customized21: Mouse, Wireless",,draft
7,SO007,Luminous Technologies,False,16: Laptop E502317: GrapWorks Software18: Datacard19: USB Adapter,,manual
6,SO006,Think Big Systems,info@thinkbig.com,15: PC Assamble + 2GB RAM,,draft
5,SO005,Agrolait,info@agrolait.com,"12: External Hard disk13: Blank DVD-RW14: Printer, All-in-one",,draft
4,SO004,Millennium Industries,False,"8: Service on demand9: Webcam10: Multimedia Speakers11: Switch, 24 ports",,draft
3,SO003,Chamber Works,info@chamberworks.com,6: On Site Monitoring7: Toner Cartridge,,draft
2,SO002,Bank Wealthy and sons,email@wealthyandsons.com,4: Service on demand5: On Site Assistance,,draft
1,SO001,Agrolait,info@agrolait.com,"1: Laptop E50232: Pen drive, 16GB3: Headset USB",,draft


There also available to_csv method, which allow to represent table in csv format

In [11]:
so_table.to_csv()

/home/katyukha/projects/erp-proxy/examples/tmp/csv/tmpFI0iVI.csv

## Access one element of recordlist via index

In [12]:
so_list[0]


Object,Object ('sale.order')
Proxy,xml-rpc://admin@localhost:8069/demo_db_1
Name,SO008


## Display one sale order as HTML Table

In [13]:
so_list[0].as_html('name',
                   'origin',
                   'partner_id',        # Will display Reacord instance representing partner related to this sale order
                   'partner_id._name',  # Will display result of 'name_get' called on partner
                   'partner_id.sale_order_ids.length')  # Display how many sale orders have this partner

Column,Value
name,SO008
origin,False
partner_id,"R(res.partner, 19)[Millennium Industries]"
partner_id._name,Millennium Industries
partner_id.sale_order_ids.length,2


In [14]:
so_list[0].as_html()  # Display all fields for firest sale order record

Column,Value
Confirmation Date,False
Contract / Analytic,False
Create Invoice,manual
Creation Date,2015-07-14 12:40:59
Customer,"R(res.partner, 19)[Millennium Industries]"
Customer Reference,False
Date,2015-07-14
Delivery Address,"R(res.partner, 52)[Millennium Industries, Jacob Taylor]"
Fiscal Position,False
Invoice Address,"R(res.partner, 52)[Millennium Industries, Jacob Taylor]"
